# First appraoches

In [7]:
from nba_api.stats.endpoints import leaguedashplayerclutch
import inspect

# Print the parameters accepted by LeagueDashPlayerClutch
print(inspect.signature(leaguedashplayerclutch.LeagueDashPlayerClutch.__init__))

(self, ahead_behind='Ahead or Behind', clutch_time='Last 5 Minutes', last_n_games='0', measure_type_detailed_defense='Base', month='0', opponent_team_id=0, pace_adjust='N', per_mode_detailed='Totals', period='0', plus_minus='N', point_diff='5', rank='N', season='2024-25', season_type_all_star='Regular Season', college_nullable='', conference_nullable='', country_nullable='', date_from_nullable='', date_to_nullable='', division_simple_nullable='', draft_pick_nullable='', draft_year_nullable='', game_scope_simple_nullable='', game_segment_nullable='', height_nullable='', league_id_nullable='', location_nullable='', outcome_nullable='', po_round_nullable='', player_experience_nullable='', player_position_abbreviation_nullable='', season_segment_nullable='', shot_clock_range_nullable='', starter_bench_nullable='', team_id_nullable='', vs_conference_nullable='', vs_division_nullable='', weight_nullable='', proxy=None, headers=None, timeout=30, get_request=True)


In [8]:
from nba_api.stats.endpoints import leaguedashplayerclutch
import pandas as pd

# Get clutch stats with the correct parameters
clutch_stats = leaguedashplayerclutch.LeagueDashPlayerClutch(
    ahead_behind='Ahead or Behind',
    clutch_time='Last 5 Minutes',
    point_diff='5',
    season='2023-24',
    season_type_all_star='Regular Season',
    measure_type_detailed_defense='Base',
    per_mode_detailed='PerGame'
)

# Convert to DataFrame
clutch_df = clutch_stats.get_data_frames()[0]

In [9]:
clutch_df.shape

(469, 67)

# Get all clutch plays and select top performers

In [22]:
from nba_api.stats.endpoints import leaguedashplayerclutch
import pandas as pd
import time
import random

def get_clutch_stats_for_season(season='2023-24', season_chunk="Regular Season", min_minutes=0, min_min = 1):
    """
    Get clutch stats for all players in a given season with no filtering
    
    Parameters:
    season (str): Season in format '2023-24'
    min_minutes (float): Minimum clutch minutes to include (0 = no filtering)
    
    Returns:
    pandas.DataFrame: DataFrame with clutch stats for all players
    """
    try:
        print(f"Fetching clutch stats for season {season}...")
        
        # Parameters for clutch time (last 5 min, game within 5 points)
        clutch_stats = leaguedashplayerclutch.LeagueDashPlayerClutch(
            season=season,
            season_type_all_star=season_chunk,
            measure_type_detailed_defense='Advanced',
            clutch_time='Last 5 Minutes',
            point_diff='5',
            per_mode_detailed='Totals'  # Get total stats instead of per game
        )
        
        # Get the data
        df = clutch_stats.get_data_frames()[0]
        
        # Force the SEASON column
        df['SEASON'] = season
        
        # Filter for minimum minutes if specified
        if min_minutes > 0:
            filtered_df = df[df['MIN'] >= min_minutes].copy()
            filtered_df = filtered_df[filtered_df['MIN'] >= min_min]
            print(f"Found {len(filtered_df)} players with ≥{min_minutes} clutch minutes in {season} (filtered from {len(df)} total)")
        else:
            filtered_df = df.copy()
            print(f"Found {len(filtered_df)} players with clutch data in {season}")
        
        return filtered_df
        
    except Exception as e:
        print(f"Error getting clutch stats for season {season}: {e}")
        import traceback
        traceback.print_exc()
        return None

def process_multiple_seasons(start_year, end_year):
    """
    Process multiple seasons of clutch data with no intermediate saving
    
    Parameters:
    start_year (int): Starting year (e.g., 2018 for 2018-19 season)
    end_year (int): Ending year (e.g., 2023 for 2023-24 season)
    
    Returns:
    pandas.DataFrame: Combined DataFrame with clutch stats for all seasons
    """
    all_seasons_data = []
    
    for year in range(start_year, end_year + 1):
        # Create season string (e.g., '2023-24')
        season = f"{year}-{str(year+1)[-2:]}"
        
        print(f"\n=== Processing {season} season ===")
        
        # Get clutch stats for this season - no minimum minutes filter
        season_data = get_clutch_stats_for_season(season, min_minutes=1)
        
        if season_data is not None and not season_data.empty:
            # Add to the combined data
            all_seasons_data.append(season_data)
            
            print(f"Added {len(season_data)} players from {season} to dataset")
            
            # Add delay between seasons to avoid rate limiting
            if year < end_year:
                delay = random.uniform(2, 4)
                print(f"Waiting {delay:.2f} seconds before next season...")
                time.sleep(delay)
        else:
            print(f"No data collected for {season}")
    
    # Combine all seasons
    if all_seasons_data:
        combined_data = pd.concat(all_seasons_data, ignore_index=True)
        print(f"\nTotal: {len(combined_data)} player-season records across {len(all_seasons_data)} seasons")
        return combined_data
    else:
        print(f"No data collected for any seasons")
        return pd.DataFrame()

def get_top_clutch_performers(df, n=50, min_games=10, min_min=1):
    """
    Get top clutch performers from a DataFrame with more robust filtering
    
    Parameters:
    df (pandas.DataFrame): DataFrame with clutch stats
    n (int): Number of top performers to return
    min_games (int): Minimum games played in clutch situations
    
    Returns:
    pandas.DataFrame: DataFrame with top clutch performers
    """
    # Filter for minimum games played
    filtered_df = df[df['GP'] >= min_games].copy()
    filtered_df = filtered_df[filtered_df['MIN'] >= min_min]

    
    print(f"Filtering for players with at least {min_games} clutch games")
    print(f"Before: {len(df)} players, After: {len(filtered_df)} players")
    
    # Sort by clutch impact score and get top n
    top_players = filtered_df.sort_values('NET_RATING', ascending=False).head(n)
    
    # Select relevant columns for display
    display_cols = [
        'PLAYER_NAME', 'TEAM_ABBREVIATION', 'SEASON', 
        'GP', 'MIN', 'NET_RATING', 'OFF_RATING', 'DEF_RATING',
        'USG_PCT', 'TS_PCT', 'PIE'
    ]
    
    # Get columns that exist in the DataFrame
    available_cols = [col for col in display_cols if col in top_players.columns]
    
    return top_players[available_cols]


In [11]:
# Example usage
# Get data for multiple seasons (with no intermediate saving)
all_data = process_multiple_seasons(2018, 2023)

if not all_data.empty:
    # Get top clutch performers across all seasons (min 10 games)
    top_performers = get_top_clutch_performers(all_data, n=50, min_games=10)
    
    print("\nTop 50 Clutch Performers (2018-2023) with at least 10 clutch games:")
    print(top_performers)
    
    # Save only the final results
    final_filename = "top_clutch_performers_2018-2023.csv"
    top_performers.to_csv(final_filename, index=False)
    print(f"Saved top performers to {final_filename}")
    
    # Also save the complete dataset
    complete_filename = "all_clutch_data_2018-2023.csv"
    all_data.to_csv(complete_filename, index=False)
    print(f"Saved complete dataset to {complete_filename}")


=== Processing 2018-19 season ===
Fetching clutch stats for season 2018-19...
Found 451 players with clutch data in 2018-19
Added 451 players from 2018-19 to dataset
Waiting 3.59 seconds before next season...

=== Processing 2019-20 season ===
Fetching clutch stats for season 2019-20...
Found 456 players with clutch data in 2019-20
Added 456 players from 2019-20 to dataset
Waiting 3.40 seconds before next season...

=== Processing 2020-21 season ===
Fetching clutch stats for season 2020-21...
Found 454 players with clutch data in 2020-21
Added 454 players from 2020-21 to dataset
Waiting 2.49 seconds before next season...

=== Processing 2021-22 season ===
Fetching clutch stats for season 2021-22...
Found 496 players with clutch data in 2021-22
Added 496 players from 2021-22 to dataset
Waiting 2.52 seconds before next season...

=== Processing 2022-23 season ===
Fetching clutch stats for season 2022-23...
Found 451 players with clutch data in 2022-23
Added 451 players from 2022-23 to d

In [12]:
all_data[all_data['PLAYER_NAME'] == 'Aaron Gordon'].groupby(['PLAYER_NAME', 'SEASON']).agg({
    'NET_RATING': 'mean',
    'GP': 'sum',
    'MIN': 'sum'
}).reset_index()

,PLAYER_NAME,SEASON,NET_RATING,GP,MIN
0,Aaron Gordon,2018-19,-0.4,43,2.9
1,Aaron Gordon,2019-20,-6.6,27,3.3
2,Aaron Gordon,2020-21,7.7,19,2.9
3,Aaron Gordon,2021-22,11.0,32,4.0
4,Aaron Gordon,2022-23,16.0,28,3.7
5,Aaron Gordon,2023-24,19.4,38,3.2


# Select Top 10 Clutch Players Based on +/-

In [32]:
# Example usage
# Get data for multiple seasons
start_year = 1990
end_year = 2025
all_data = process_multiple_seasons(start_year=start_year, end_year=end_year)

if not all_data.empty:
    # Get top clutch performers across all seasons (min 10 games)
    top_performers = get_top_clutch_performers(all_data, n=10, min_games=10, min_min=2.5)
    
    print(f"\nTop 10 Clutch Performers (2000-2023) with at least 10 clutch games:")
    print(top_performers)
    
    # Save only the final results
    final_filename = f"top_clutch_performers_{start_year}_{end_year}.csv"
    top_performers.to_csv(final_filename, index=False)
    print(f"Saved top performers to {final_filename}")
    
    # # Also save the complete dataset
    complete_filename = f"all_clutch_data_{start_year}-{end_year}.csv"
    all_data.to_csv(complete_filename, index=False)
    print(f"Saved complete dataset to {complete_filename}")


=== Processing 1990-91 season ===
Fetching clutch stats for season 1990-91...
Error getting clutch stats for season 1990-91: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)
No data collected for 1990-91

=== Processing 1991-92 season ===
Fetching clutch stats for season 1991-92...


Traceback (most recent call last):
  File "/Users/estebansanchez/Desktop/nba_clutch_players_202503/.nba_clutch/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/Users/estebansanchez/Desktop/nba_clutch_players_202503/.nba_clutch/lib/python3.12/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readl

Found 0 players with ≥1 clutch minutes in 1991-92 (filtered from 0 total)
No data collected for 1991-92

=== Processing 1992-93 season ===
Fetching clutch stats for season 1992-93...
Found 0 players with ≥1 clutch minutes in 1992-93 (filtered from 0 total)
No data collected for 1992-93

=== Processing 1993-94 season ===
Fetching clutch stats for season 1993-94...
Found 0 players with ≥1 clutch minutes in 1993-94 (filtered from 0 total)
No data collected for 1993-94

=== Processing 1994-95 season ===
Fetching clutch stats for season 1994-95...
Found 0 players with ≥1 clutch minutes in 1994-95 (filtered from 0 total)
No data collected for 1994-95

=== Processing 1995-96 season ===
Fetching clutch stats for season 1995-96...
Found 0 players with ≥1 clutch minutes in 1995-96 (filtered from 0 total)
No data collected for 1995-96

=== Processing 1996-97 season ===
Fetching clutch stats for season 1996-97...
Found 343 players with ≥1 clutch minutes in 1996-97 (filtered from 393 total)
Added 3

In [35]:
top_performers

,PLAYER_NAME,TEAM_ABBREVIATION,SEASON,GP,MIN,NET_RATING,OFF_RATING,DEF_RATING,USG_PCT,TS_PCT,PIE
9281,Chris Paul,PHX,2021-22,29,3.1,52.0,138.4,86.4,0.303,0.689,0.300
7670,Chris Paul,HOU,2017-18,21,2.6,50.1,145.5,95.5,0.324,0.797,0.302
6222,Jared Dudley,LAC,2013-14,17,3.2,48.0,126.7,78.7,0.071,0.788,0.082
6359,Patty Mills,SAS,2013-14,13,3.6,47.6,119.2,71.6,0.223,0.558,0.193
4595,Zydrunas Ilgauskas,CLE,2008-09,18,2.5,47.2,135.9,88.6,0.165,0.716,0.070
7096,Mario Hezonja,ORL,2015-16,11,2.8,46.6,116.4,69.8,0.263,0.766,0.206
10589,Evan Mobley,CLE,2024-25,22,2.9,46.0,148.3,102.2,0.121,0.799,0.118
5828,Greg Smith,HOU,2012-13,10,2.8,45.4,118.8,73.3,0.097,0.962,0.261
9148,Robin Lopez,WAS,2020-21,11,3.8,45.0,129.7,84.7,0.147,0.795,0.178
8957,Gordon Hayward,CHA,2020-21,20,3.1,44.9,138.0,93.1,0.272,0.606,0.213


# Analyze Players to pin point attributes

In [36]:
print(f'Players: {set(player for player in top_performers.PLAYER_NAME.values)}')

Players: {'Zydrunas Ilgauskas', 'Robin Lopez', 'Jared Dudley', 'Greg Smith', 'Patty Mills', 'Chris Paul', 'Mario Hezonja', 'Evan Mobley', 'Gordon Hayward'}


### `Chris Paul`


Lets begin with someone who appears twice on the list. Lets see what makes CP3 such a clutch player. First of all, lets analyze how good he actually is, and compare his `NET_RATING` to more time than just clutch minutes. Also we should compare to the playoffs. 

### `Zydrunas Ilgauskas`


### `Robin Lopez`


### `Jared Dudley`


### `Greg Smith`


### `Patty Mills`


### `Evan Mobley`


### `Gordon Hayward`


### `Mario Hezonja`